In [1]:
import torch
from brt._C.router import (
    generate_indices_and_loads,
    dispatch_with_indices_and_loads,
    split_fused_cells_to_paths,
    fuse_split_cells_from_paths,
    combine_with_indices_and_loads,
)

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_tag_index=False
)
print(route_indices)
print(dst_loads)


/home/whcui/.pyenv/versions/miniconda3-3.8-4.11.0/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[0, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [2, 0, 0, 0],
        [0, 0, 0, 2],
        [0, 0, 1, 0],
        [0, 0, 2, 0],
        [0, 0, 3, 0]], device='cuda:0', dtype=torch.int32)
tensor([2, 1, 4, 2], device='cuda:0', dtype=torch.int32)


In [3]:
in_data = torch.randn((8, path_num)).cuda()
print(in_data)
print(route_indices)
print(dst_loads)
# (out_data_1,) = dispatch_with_indices_and_loads(in_data, route_indices, dst_loads)
# print(out_data_1)
# split_data = split_fused_cells_to_paths(out_data_1, dst_loads)
# print(split_data)

out_data_1, tags = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, tag_generating=True
)
print(tags)

split_data, split_loads, split_tags = split_fused_cells_to_paths(
    out_data_1, dst_loads, is_load_split=True, is_tag_split=True, tags=tags
)
print(split_tags)


# results = (
#     torch.zeros((9, 4), dtype=torch.float32)
#     .cuda()
#     .scatter_reduce(0, global_dst_indices, fused_data, reduce="sum")
# )

# out_data_1, tags = dispatch_with_indices_and_loads(
#     in_data, route_indices, dst_loads, max_path_padding=True, tag_generating=True
# )
# split_data = split_fused_cells_to_paths(out_data_1, dst_loads, max_path_padding=True,tags=tags)
# print(split_data)# print(out_data_1)

# out_data_2 = dispatch_with_indices_and_loads(
#     in_data, route_indices, dst_loads, is_1d_routing=False
# )
# print(out_data_2)
# torch.allclose(out_data_1[0], out_data_2)


tensor([[-0.2296, -0.7083, -0.2889,  0.0953],
        [-0.4906, -1.1487,  0.6038,  0.0545],
        [-0.3319, -1.3258,  0.9305,  1.7306],
        [ 0.0906,  0.2670,  0.4063, -0.3558],
        [-1.0018, -0.3968, -0.2029, -0.0085],
        [ 0.6007, -0.9677, -0.1781, -0.4924],
        [-1.1791,  1.4208,  0.1571,  0.5924],
        [-1.5516, -0.7332, -0.5385, -0.6513]], device='cuda:0')
tensor([[0, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [2, 0, 0, 0],
        [0, 0, 0, 2],
        [0, 0, 1, 0],
        [0, 0, 2, 0],
        [0, 0, 3, 0]], device='cuda:0', dtype=torch.int32)
tensor([2, 1, 4, 2], device='cuda:0', dtype=torch.int32)
tensor([3, 4, 1, 6, 7, 8, 0, 2, 5], device='cuda:0', dtype=torch.int32)
[tensor([3, 4], device='cuda:0', dtype=torch.int32), tensor([1], device='cuda:0', dtype=torch.int32), tensor([6, 7, 8, 0], device='cuda:0', dtype=torch.int32), tensor([2, 5], device='cuda:0', dtype=torch.int32)]


In [4]:
fused_data, new_tags, global_seat_indices = fuse_split_cells_from_paths(
    split_data, is_tag_fuse=True, tags=split_tags
)

print(fused_data)
print(new_tags)
print(global_seat_indices)

final_data = combine_with_indices_and_loads(
    fused_data, global_seat_indices, dst_loads, tags=new_tags, is_tag_index=True
)
print(final_data)
print(torch.allclose(final_data, in_data))





tensor([[-0.3319, -1.3258,  0.9305,  1.7306],
        [ 0.0906,  0.2670,  0.4063, -0.3558],
        [-0.2296, -0.7083, -0.2889,  0.0953],
        [ 0.6007, -0.9677, -0.1781, -0.4924],
        [-1.1791,  1.4208,  0.1571,  0.5924],
        [-1.5516, -0.7332, -0.5385, -0.6513],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4906, -1.1487,  0.6038,  0.0545],
        [-1.0018, -0.3968, -0.2029, -0.0085]], device='cuda:0')
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8], device='cuda:0', dtype=torch.int32)
tensor([3, 4, 1, 6, 7, 8, 0, 2, 5], device='cuda:0')
tensor([[-0.2296, -0.7083, -0.2889,  0.0953],
        [-0.4906, -1.1487,  0.6038,  0.0545],
        [-0.3319, -1.3258,  0.9305,  1.7306],
        [ 0.0906,  0.2670,  0.4063, -0.3558],
        [-1.0018, -0.3968, -0.2029, -0.0085],
        [ 0.6007, -0.9677, -0.1781, -0.4924],
        [-1.1791,  1.4208,  0.1571,  0.5924],
        [-1.5516, -0.7332, -0.5385, -0.6513]], device='cuda:0')
True


[W TensorAdvancedIndexing.cpp:1550] Warning: scatter_reduce() is in beta and the API may change at any time. (function operator())


In [5]:
fused_data, fused_loads = fuse_split_cells_from_paths(
    split_data, is_load_fuse=True, loads=split_loads
)

print(fused_data)
print(new_tags)
print(global_seat_indices)

final_data = combine_with_indices_and_loads(
    fused_data, route_indices, fused_loads
)
print(final_data)
print(torch.allclose(final_data, in_data))


tensor([[-0.3319, -1.3258,  0.9305,  1.7306],
        [ 0.0906,  0.2670,  0.4063, -0.3558],
        [-0.2296, -0.7083, -0.2889,  0.0953],
        [ 0.6007, -0.9677, -0.1781, -0.4924],
        [-1.1791,  1.4208,  0.1571,  0.5924],
        [-1.5516, -0.7332, -0.5385, -0.6513],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4906, -1.1487,  0.6038,  0.0545],
        [-1.0018, -0.3968, -0.2029, -0.0085]], device='cuda:0')
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8], device='cuda:0', dtype=torch.int32)
tensor([3, 4, 1, 6, 7, 8, 0, 2, 5], device='cuda:0')
tensor([[-0.2296, -0.7083, -0.2889,  0.0953],
        [-0.4906, -1.1487,  0.6038,  0.0545],
        [-0.3319, -1.3258,  0.9305,  1.7306],
        [ 0.0906,  0.2670,  0.4063, -0.3558],
        [-1.0018, -0.3968, -0.2029, -0.0085],
        [ 0.6007, -0.9677, -0.1781, -0.4924],
        [-1.1791,  1.4208,  0.1571,  0.5924],
        [-1.5516, -0.7332, -0.5385, -0.6513]], device='cuda:0')
True
